In [ ]:
#import modules
import pandas as pd
import numpy as np 
import seaborn as sb
import pandas_profiling
import matplotlib.pyplot as plt
import sqlalchemy as db
from sqlalchemy import MetaData, Table
from pandas_profiling import ProfileReport

In [ ]:
#connecting to database
engine = db.create_engine("sqlite:///im.db")

In [ ]:
#getting all table names
inspector = db.inspect(engine)
table_names = inspector.get_table_names()

In [ ]:
metadata = MetaData()
connection = engine.connect()
all_data = []

#create for loop to get lists from tables
for i in table_names:
    t = Table(i, metadata, autoload = True, autoload_with=engine )
    query = db.select([t])
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    all_data.append(ResultSet)

In [ ]:
# creating dataframes
directors = pd.DataFrame(all_data[0])
known_for = pd.DataFrame(all_data[1])
movie_akas = pd.DataFrame(all_data[2])
movie_basics = pd.DataFrame(all_data[3])
movie_ratings = pd.DataFrame(all_data[4])
persons = pd.DataFrame(all_data[5])
principals = pd.DataFrame(all_data[6])
writers = pd.DataFrame(all_data[7])


In [ ]:
# creating profile report
profile = ProfileReport(movie_basics)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import autoviz

ModuleNotFoundError: No module named 'autoviz'